In [1]:
import pandas as pd
import re
import string
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
true=pd.read_csv('/Users/rahulahirwar/Desktop/FND/True.csv')
true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
fake=pd.read_csv('/Users/rahulahirwar/Desktop/FND/Fake.csv')

In [5]:
true.insert(4,'label',1)

In [6]:
fake.insert(4,'label',0)

In [7]:
news=pd.concat([true,fake],axis=0)

In [8]:
news.isnull().any()

,0
title,False
text,False
subject,False
date,False
label,False


In [9]:
news=news.drop(['subject','title','date'],axis=1)

In [10]:
news=news.sample(frac=1)

In [11]:
news.reset_index(inplace=True)

In [12]:
news.drop(['index'],axis=1,inplace=True)

In [16]:
def wordopt(text):
    tt = text.lower()
    t = re.sub('\[.*?\]', '', text)
    t = re.sub("\\W"," ",text)
    t = re.sub('https?://\S+|www\.\S+', '', text)
    t = re.sub('<.*?>+', '', text)
    t = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    t = re.sub('\n', '', text)
    t = re.sub('\w*\d\w*', '', text)
    return text

news["text"] = news["text"].apply(wordopt)


x = news["text"]
y = news["label"]

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
x_train = vectorization.fit_transform(x_train)
x_test = vectorization.transform(x_test)


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [21]:
prediction = lr.predict(x_test)
lr.score(x_test,y_test)

0.9861024498886414

In [22]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5782
           1       0.98      0.99      0.99      5443

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [25]:
from joblib import dump
joblib.dump(vectorization,"vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

['lr_model.jb']